#### Load required modules

In [21]:
import mlflow.pyfunc
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#### Load dataset

In [22]:
iris = load_iris()
iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_train.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


#### Train and evaluate a random forest classifier model

In [23]:
# train
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(iris_train, iris.target)

# evaluate
iris_pred = clf.predict(iris_train)
accuracy = accuracy_score(iris.target, iris_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 100.00%


#### Get or create MLFlow experiment and log model to model registery

In [24]:
experiment_name = "iris-experiment-v2"

try:
    mlflow.set_experiment(experiment_name)
except:
    mlflow.create_experiment(experiment_name)

with mlflow.start_run():
    mlflow.log_param("max_depth", 7)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(clf, "iris_rf")

#### Get the experiment

In [25]:
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

print("Experiment ID:", experiment_id, "Experiment NAME:", experiment_name)

Experiment ID: 949272421715945670 Experiment NAME: iris-experiment-v2


#### Search for runs that match a set of criteria

In [26]:
runs = mlflow.search_runs(experiment_ids=[experiment_id])

for index, run in runs.iterrows():
    print(run['run_id'], run['start_time'], run['status'])

e9f75726ca594ab4b8260b66ff738b5e 2023-04-06 21:54:08.038000+00:00 FINISHED
54ae8336e43049a1a537657ad544894b 2023-04-06 21:38:50.652000+00:00 FINISHED


#### Get model to make predictions

In [27]:
# get model
model_uri = r"mlruns/{}/{}/artifacts/iris_rf".format(experiment_id, runs['run_id'][0])
model = mlflow.pyfunc.load_model(model_uri)

# make predictions using the loaded model
predictions = model.predict(iris_train[1:3])

# print the predictions
print(predictions)

[0 0]


#### Use Model Registry to manage the lifecycle of models

In [28]:
# register a new model version
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id
runs = mlflow.search_runs(experiment_ids=[experiment_id])

model_uri = "runs:/{}/iris_rf".format(runs['run_id'][0])
last_run = runs.sort_values(by='end_time', ascending=False)

if last_run['metrics.accuracy'][0] > 0.7:
    model_version = mlflow.register_model(model_uri, "iris_rf_model")


# Load a model from MLflow
model_uri = "models:/iris_rf_model/{}".format(model_version.version)
model = mlflow.pyfunc.load_model(model_uri)
model

Registered model 'iris_rf_model' already exists. Creating a new version of this model...
2023/04/06 23:54:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris_rf_model, version 3
Created version '3' of model 'iris_rf_model'.


mlflow.pyfunc.loaded_model:
  artifact_path: iris_rf
  flavor: mlflow.sklearn
  run_id: e9f75726ca594ab4b8260b66ff738b5e

#### Visualizing and Analyzing Experiments and Models with the MLflow UI

In [29]:
!mlflow ui --backend-store-uri file:mlruns --port 8908

[2023-04-06 23:54:23 +0200] [13107] [INFO] Starting gunicorn 20.1.0
[2023-04-06 23:54:23 +0200] [13107] [INFO] Listening at: http://127.0.0.1:8908 (13107)
[2023-04-06 23:54:23 +0200] [13107] [INFO] Using worker: sync
[2023-04-06 23:54:23 +0200] [13108] [INFO] Booting worker with pid: 13108
[2023-04-06 23:54:23 +0200] [13109] [INFO] Booting worker with pid: 13109
[2023-04-06 23:54:23 +0200] [13110] [INFO] Booting worker with pid: 13110
[2023-04-06 23:54:23 +0200] [13111] [INFO] Booting worker with pid: 13111
^C
[2023-04-06 23:54:26 +0200] [13107] [INFO] Handling signal: int
[2023-04-06 23:54:26 +0200] [13111] [INFO] Worker exiting (pid: 13111)
[2023-04-06 23:54:26 +0200] [13110] [INFO] Worker exiting (pid: 13110)
[2023-04-06 23:54:26 +0200] [13109] [INFO] Worker exiting (pid: 13109)
[2023-04-06 23:54:26 +0200] [13108] [INFO] Worker exiting (pid: 13108)
